In [189]:
%load_ext autoreload
%autoreload 2

# pip install easydict
import easydict
import torch
import torch.nn as nn
import torch.nn.functional as F

from data import MetaLearningSystemDataLoader
from data import FewShotLearningDatasetParallel
from utils.parser_utils import get_args


from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12, extract_top_level_dict, MetaBatchNormLayer, MetaLinearLayer
from inner_loop_optimizers import LSLRGradientDescentLearningRule

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [190]:
args = easydict.EasyDict(
{
  "batch_size":2,
  "image_height":84,
  "image_width":84,
  "image_channels":3,
  "gpu_to_use":0,
  "num_dataprovider_workers":4,
  "max_models_to_save":5,
  "dataset_name":"mini_imagenet_full_size",
  "dataset_path":"mini_imagenet_full_size",
  "reset_stored_paths":False,
  "experiment_name":"alfa+maml",
  "train_seed": 0, "val_seed": 0,
  "indexes_of_folders_indicating_class": [-3, -2],
  "sets_are_pre_split": True,
  "train_val_test_split": [0.64, 0.16, 0.20],
  "evaluate_on_test_set_only": False,

  "total_epochs": 100,
  "total_iter_per_epoch":500, "continue_from_epoch": -2,
  "num_evaluation_tasks":600,
  "multi_step_loss_num_epochs": 15,
  "minimum_per_task_contribution": 0.01,
  "learnable_per_layer_per_step_inner_loop_learning_rate": False,
  "enable_inner_loop_optimizable_bn_params": False,
  "evalute_on_test_set_only": False,

  "max_pooling": True,
  "per_step_bn_statistics": False,
  "learnable_batch_norm_momentum": False,
  "load_into_memory": False,
  "init_inner_loop_learning_rate": 0.01,
  "init_inner_loop_weight_decay": 0.0005,
  "learnable_bn_gamma": True,
  "learnable_bn_beta": True,

  "dropout_rate_value":0.0,
  "min_learning_rate":0.001,
  "meta_learning_rate":0.001,   "total_epochs_before_pause": 100,
  "first_order_to_second_order_epoch":-1,
  "weight_decay": 0.0,

  "norm_layer":"batch_norm",
  "cnn_num_filters":48,
  "num_stages":4,
  "conv_padding": True,
  "number_of_training_steps_per_iter":5,
  "number_of_evaluation_steps_per_iter":5,
  "cnn_blocks_per_stage":1,
  "num_classes_per_set":5,
  "num_samples_per_class":5,
  "num_target_samples": 15,

  "second_order": True,
  "use_multi_step_loss_optimization":False,
  "attenuate": False,
  "alfa": True,
  "random_init": False,
  "backbone": "4-CONV"
}
)

device = torch.cuda.current_device()
im_shape = (2, 3, args.image_height, args.image_width)

args.use_cuda = torch.cuda.is_available()
args.seed = 104
args.reverse_channels=False
args.labels_as_int=False
args.reset_stored_filepaths=False
args.num_of_gpus=1

In [191]:
data = MetaLearningSystemDataLoader

In [192]:
input_dim = 10
input_shape = (1, input_dim)

meta_linear = MetaLinearLayer(input_shape=input_shape, num_filters=input_dim, use_bias=True)

In [193]:
for name, param in meta_linear.named_parameters():
    print(name, param.shape)

weights torch.Size([10, 10])
bias torch.Size([10])


In [194]:
x1 = torch.tensor([1, 2, 3, 4])
x2 = torch.tensor([5, 6, 7, 8])
x3 = torch.tensor([9, 10, 11 ,12])

a = []
a.append(x1)
a.append(x2)
a.append(x3)

print("a len == ",len(a))

# inputs = torch.stack(a)
inputs = torch.cat(a)
inputs


a len ==  3


tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [195]:
input_dim = 10
input_shape = (1, input_dim)
num_filters=input_dim
b, c = input_shape #c=10

weights1 = nn.Parameter(torch.ones(num_filters, c))
# weight = nn.Parameter(torch.FloatTensor(out_features, in_features))

nn.init.xavier_uniform_(weights1)
#weights

Parameter containing:
tensor([[-0.3419, -0.1161, -0.0592,  0.5328, -0.5382,  0.0276, -0.4067,  0.1050,
          0.2646,  0.1206],
        [-0.2462, -0.5362, -0.1066,  0.0017, -0.4400, -0.1472, -0.3142,  0.1492,
         -0.1956, -0.1493],
        [ 0.3184,  0.4922,  0.1096, -0.3266,  0.3313,  0.3086, -0.4515, -0.0245,
          0.3348,  0.2542],
        [-0.1074, -0.4810, -0.0694,  0.1268, -0.0351, -0.3420,  0.3856, -0.0643,
         -0.4052, -0.1178],
        [ 0.2284, -0.2137, -0.4187, -0.1576, -0.3951, -0.2103,  0.0924, -0.0915,
          0.1532,  0.4606],
        [-0.0051,  0.1155,  0.2027,  0.4556,  0.3564, -0.1232, -0.3023, -0.2142,
          0.4530, -0.0949],
        [-0.0444, -0.0790,  0.2115,  0.3245,  0.2490, -0.5222,  0.3717, -0.5193,
         -0.1560,  0.3584],
        [ 0.3703,  0.4295,  0.0900,  0.5290,  0.2163,  0.1366,  0.0608,  0.1772,
          0.0112,  0.2723],
        [-0.0563,  0.3309, -0.3247,  0.2516, -0.5091, -0.4323,  0.5211,  0.4224,
         -0.2842, -0.1994

In [196]:
bias1 = nn.Parameter(torch.zeros(num_filters))
bias1

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [197]:
x = torch.rand(3, 10)

out1 = F.linear(input=x, weight=weights1, bias=bias1)
# y = xW^T + b

out1

tensor([[ 0.0186, -1.0784,  0.9117, -0.8201, -0.1632,  0.7538,  0.0961,  1.0518,
         -0.3233,  0.3555],
        [-0.5200, -0.6977,  0.6746, -0.4093, -0.4079,  0.0644, -0.1693,  0.7819,
         -0.5199,  0.5876],
        [-0.3547, -1.0406,  0.8629, -0.5238, -0.0830,  0.4208,  0.5054,  1.1539,
         -0.2066,  0.0055]], grad_fn=<AddmmBackward0>)

In [198]:
weights2 = nn.Parameter(torch.ones(2, 10))
nn.init.xavier_uniform_(weights2)
out2 = F.linear(input=out1, weight=weights2)
# y = x*W^T + b

out2

tensor([[0.6955, 1.9939],
        [0.2063, 1.4041],
        [0.4751, 1.7669]], grad_fn=<MmBackward0>)

In [199]:
out3 = F.relu_(out2)
out3

tensor([[0.6955, 1.9939],
        [0.2063, 1.4041],
        [0.4751, 1.7669]], grad_fn=<ReluBackward0>)